# MIDI INPUT AND OUTPUT

An introduction to reading and writing MIDI data to and from files.

<hr style="height:1px;color:gray">

Notebook imports:

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from musx import Score, Note, MidiEvent, Seq, MidiFile, odds, rhythm, version, \
pitch, histo, between, setmidiplayer, playfile
from musx import midi
from musx.midi.gm import HandClap, Maracas, Cowbell, HiWoodBlock, LowBongo, LowWoodBlock
print(f"musx.version: {version}")

This notebook generates MIDI files and automatically plays them using [fluidsynth](https://www.fluidsynth.org/download/) and the [MuseScore_General.sf3](https://ftp.osuosl.org/pub/musescore/soundfont/MuseScore_General) sound font. See [INSTALL.md](https://github.com/musx-admin/musx/blob/main/INSTALL.md) for how to install a terminal-based MIDI player to use with musx.  If you don't have a player installed you can access the output files in the same directory as this notebook:

In [ ]:
setmidiplayer("fluidsynth -iq -g1 /Users/taube/Music/SoundFonts/MuseScore_General.sf2")
print('OK!')

### Midi Messages

musx provides both a low level and a high level of midi support.  The lowest level exports functions that create or access MIDI messages represented as python lists, where each element in the list is a MIDI data byte.  These low level MIDI messages can be written directly to files, or sent in real time to an external MIDI port (see rtmidi.ipynb):

Note: In this notebook the midi module's functions are prefixed with the module name, e.g. midi.note_on().

In [ ]:
a = midi.note_on(4, 60, 100)
print("note on:", a)
print("note on channel:", midi.channel(a))
print("note on keynum:", midi.keynum(a))
print("note on velocity:", midi.velocity(a))
print("is channel message:", midi.is_channel_message(a))

The higher level API implements an object-oriented MidiEvent class that associates a MIDI time stamp with a low-level MIDI message. The MidiEvent class also provides event constructors, accessors and predicates for every type of MIDI message, including Meta Messages:

In [ ]:
a = midi.MidiEvent.note_on(4, 60, 100, time=9.3)
print(a)
print("note on channel:", a.channel())
print("note on keynum:", a.keynum())
print("note on velocity:", a.velocity())
print("note on time:", a.time)

### Loading MidiEvents from a MIDI file

To load MIDI events from a file use the `midi.MidiFile.read()` method. The result of the call will be a MidiFile object with one or more MIDI tracks, each represented by a musx `Seq` object containing a list of MidiEvent instances.

In [ ]:
sotb = midi.MidiFile("./support/Song_on_the_Beach.mid").read()
print(sotb)

Since MIDI tracks are often quite large, it is always a good idea to check the length and end time of the track data before you start manipulating it. Track sequences are stored in the MidiFile.tracks attribute:

In [ ]:
sotb.tracks

### Inspecting sequence data

To inspect the objects in a sequence use the `Seq.print(start=0, end=None)` method, which provides start and end parameters so the caller can control exactly where printing should occur in the list of events:

In [ ]:
seq1 = sotb.tracks[0]
seq1.print(0, 15)

### Accessing MidiEvents in sequences

A seq is also a Python iterator so it is easy loop over its events to perform a task. This comprehension collect all the MIDI Meta Messages in the track:

In [ ]:
meta = [x for x in seq1 if x.is_meta()]
print(meta)

This example creates a list that contains only the NoteOn messages in the track:

In [ ]:
notes = [x for x in seq1 if x.is_note_on()]
print(f"Number of note on messages:", len(notes))

Given note on events its east to ascertain information about the musical information extracted from  the midifile.  This example computes a sorted histogram of all the unique notes in the track:

In [ ]:
unique = histo( sorted([str(pitch(x.keynum())) for x in notes]) )
print(unique)

### Adding MidiEvents to sequences

The most common way to create MIDI events is to compose a score of Note objects, then add the score's seq to a MidiFile so the notes are automatically converted into MidiEvents and added to the MidiFile.

It is also possible to write MidiEvents directly to Seq objects. The `Seq.add()` method automatically adds incoming events according to their start time, thus keeping the sequence in correctly sorted time ordered:

In [ ]:
myseq = Seq()

for i in range(10):
    chan = between(0, 16)
    key = between(48, 84) 
    amp = 80
    # random start times
    time = between(0.0, 20.0)
    # add a note on at a random time
    myseq.add(MidiEvent.note_on(chan, key, 100, time))
    # add its paired note off two seconds later
    myseq.add(MidiEvent.note_off(chan, key, 127, time+2.0))

print(f"{myseq}\n")
myseq.print()

Once all the events have been added, the sequence can be added to a MidiFile so it can save the data to a file on disk:

In [ ]:
file = MidiFile("midio.mid", [myseq]).write()
print(f"Wrote '{file.pathname}'.")
playfile(file.pathname)